In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import itertools
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

In [40]:
# Load the dataset
file_path = 'D:\Download\Documents\Semester 7\ML\Week 10\Dataset\Student Performance\student-por.csv'
data = pd.read_csv(file_path, sep=';')

# Display the first few rows of the dataset
data.head()

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\user\AppData\Local\Temp\ipykernel_21540\2773329279.py:2: SyntaxWarning: invalid escape sequence '\D'
  file_path = 'D:\Download\Documents\Semester 7\ML\Week 10\Dataset\Student Performance\student-por.csv'


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [41]:
# Preprocessing
label_cols = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 
              'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 
              'nursery', 'higher', 'internet', 'romantic']
numeric_cols = list(set(data.columns) - set(label_cols) - {'G3'})

# Encode categorical columns
for col in label_cols:
    data[col] = LabelEncoder().fit_transform(data[col])

# Standardize numeric columns
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

# Prepare features and target
X = data.drop('G3', axis=1).values
y = data['G3'].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Define MLP model
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLPModel, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_size, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(activation_fn())
        layers.append(nn.Linear(hidden_layers[-1], 20))  # Output layer for 20 classes (G3 has 0-20 possible values)
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Experiment hyperparameters
hidden_layers_options = [[4], [8, 4], [16, 8, 4]]
activation_options = [nn.Identity, nn.Sigmoid, nn.ReLU, nn.Softmax, nn.Tanh]
epoch_options = [1, 10, 25, 50, 100, 250]
learning_rate_options = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_size_options = [16, 32, 64, 128, 256, 512]    

In [43]:
# Perform grid search
results = []
for hidden_layers, activation_fn, epochs, lr, batch_size in itertools.product(
        hidden_layers_options, activation_options, epoch_options, learning_rate_options, batch_size_options):
    
    # Create DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    # Initialize model, loss, and optimizer
    model = MLPModel(input_size=X_train.shape[1], hidden_layers=hidden_layers, activation_fn=activation_fn)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Evaluate model
    model.eval()
    y_pred = []
    with torch.no_grad():
        for X_batch, _ in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.numpy())

    accuracy = accuracy_score(y_test, y_pred)
    results.append((hidden_layers, activation_fn.__name__, epochs, lr, batch_size, accuracy))
    print(f"Hidden Layers: {hidden_layers}, Activation: {activation_fn.__name__}, Epochs: {epochs}, "
          f"LR: {lr}, Batch Size: {batch_size}, Accuracy: {accuracy:.4f}")

# Find best configuration
best_config = max(results, key=lambda x: x[-1])
print("Best Configuration:", best_config)

Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 10, Batch Size: 16, Accuracy: 0.0615
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 10, Batch Size: 32, Accuracy: 0.1385
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 10, Batch Size: 64, Accuracy: 0.1462
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 10, Batch Size: 128, Accuracy: 0.1077
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 10, Batch Size: 256, Accuracy: 0.1615
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 10, Batch Size: 512, Accuracy: 0.0231
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 1, Batch Size: 16, Accuracy: 0.1154
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 1, Batch Size: 32, Accuracy: 0.1615
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 1, Batch Size: 64, Accuracy: 0.1538
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 1, Batch Size: 128, Accuracy: 0.1385
Hidden Layers: [4], Activation: Identity, Epochs: 1, LR: 1, 

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs

Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 1, Batch Size: 32, Accuracy: 0.1923
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 1, Batch Size: 64, Accuracy: 0.0538
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 1, Batch Size: 128, Accuracy: 0.0538
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 1, Batch Size: 256, Accuracy: 0.1231
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 1, Batch Size: 512, Accuracy: 0.0923
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.1, Batch Size: 16, Accuracy: 0.2308
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.1, Batch Size: 32, Accuracy: 0.1308
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.1, Batch Size: 64, Accuracy: 0.1923
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.1, Batch Size: 128, Accuracy: 0.1000
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.1, Batch Size: 256, Accuracy: 0.1462


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs

Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.1, Batch Size: 512, Accuracy: 0.1000
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.01, Batch Size: 16, Accuracy: 0.1923
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.01, Batch Size: 32, Accuracy: 0.1000
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.01, Batch Size: 64, Accuracy: 0.0923
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.01, Batch Size: 128, Accuracy: 0.0923
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.01, Batch Size: 256, Accuracy: 0.0769
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.01, Batch Size: 512, Accuracy: 0.0923
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.001, Batch Size: 16, Accuracy: 0.1154
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.001, Batch Size: 32, Accuracy: 0.0000
Hidden Layers: [4], Activation: Softmax, Epochs: 1, LR: 0.001, Batch Size: 64, Accuracy: 0.1923
Hidden Layers: [4], Activation: Softmax, Epo

...

Hidden Layers: [16, 8, 4], Activation: Softmax, Epochs: 250, LR: 0.0001, Batch Size: 128, Accuracy: 0.1000


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Hidden Layers: [16, 8, 4], Activation: Softmax, Epochs: 250, LR: 0.0001, Batch Size: 256, Accuracy: 0.0154


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Hidden Layers: [16, 8, 4], Activation: Softmax, Epochs: 250, LR: 0.0001, Batch Size: 512, Accuracy: 0.0538
Hidden Layers: [16, 8, 4], Activation: Tanh, Epochs: 1, LR: 10, Batch Size: 16, Accuracy: 0.1308
Hidden Layers: [16, 8, 4], Activation: Tanh, Epochs: 1, LR: 10, Batch Size: 32, Accuracy: 0.1308
Hidden Layers: [16, 8, 4], Activation: Tanh, Epochs: 1, LR: 10, Batch Size: 64, Accuracy: 0.0154
Hidden Layers: [16, 8, 4], Activation: Tanh, Epochs: 1, LR: 10, Batch Size: 128, Accuracy: 0.1077
Hidden Layers: [16, 8, 4], Activation: Tanh, Epochs: 1, LR: 10, Batch Size: 256, Accuracy: 0.1000
Hidden Layers: [16, 8, 4], Activation: Tanh, Epochs: 1, LR: 10, Batch Size: 512, Accuracy: 0.0077
Hidden Layers: [16, 8, 4], Activation: Tanh, Epochs: 1, LR: 1, Batch Size: 16, Accuracy: 0.1923
Hidden Layers: [16, 8, 4], Activation: Tanh, Epochs: 1, LR: 1, Batch Size: 32, Accuracy: 0.0385
Hidden Layers: [16, 8, 4], Activation: Tanh, Epochs: 1, LR: 1, Batch Size: 64, Accuracy: 0.1923
Hidden Layers: [16, 